# Eye of the Emergency

## Libraries and dataset import

In [1]:
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import string
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\utile\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\utile\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\utile\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\utile\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
train_df = pd.read_csv('train_tweets.csv')
test_df = pd.read_csv('test_tweets.csv')

## Data Preprocess

In [3]:
unique_count = pd.DataFrame(pd.value_counts(train_df.text))
unique_count = unique_count.reset_index()
unique_count.columns = ['text', 'count']

In [4]:
train_df_unique = pd.DataFrame(train_df.text.unique(), columns = ['text'])

In [5]:
train_df_unique = train_df_unique.merge(unique_count)

In [6]:
train_df_count = train_df.merge(unique_count)

In [7]:
lg = pd.DataFrame([len(txt) for txt in train_df.text], columns = ['len_text'])

In [8]:
train_df_len = pd.concat([train_df_count, lg], axis = 1)

### Location Encoder

In [9]:
encode_location = pd.DataFrame([int(type(ct)==str) for ct in train_df_len.location], columns=['location_enc'])

In [10]:
train_df_location = pd.concat([train_df_len, encode_location], axis=1)

####  Construisez votre propre pipeline de traitement de texte 

In [11]:
df = train_df_location.copy()

In [12]:
df2 = train_df_location.copy()

##### Tokenization

In [13]:
#df['text_tokens'] = df['text'].apply(lambda x: word_tokenize(x))


In [14]:
#df.head()

#### Suppression des stops words

In [15]:
#stop_words = set(stopwords.words('english'))

In [16]:
#df['text_tokens'] = df['text_tokens'].apply(lambda x: [word for word in x if word.lower() not in stop_words])
#df.head()

In [17]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [18]:
#df['text_tokens'] = df['text_tokens'].apply(lambda x: [word for word in x if word not in string.punctuation])

In [19]:
#lemmatizer = WordNetLemmatizer()


In [20]:
#df['text_tokens'] = df['text_tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [21]:
#[list for list in df.text_tokens]

In [22]:
#df

In [23]:
# text cleaning


def text_processing(text):
     #Charger les stop-words en anglais
    stop_words = set(stopwords.words('english'))

    # Initialiser le lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Appliquer la tokenisation à tous les textes
    tokens = word_tokenize(text.lower())

    # Supprimer les ponctuations
    tokens = [word for word in tokens if word not in string.punctuation]
    

    # Supprimer les stop-words
    tokens = [word for word in tokens if word not in stop_words]
    

    # Appliquer la lemmatisation à tous les tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    tokens = ' '.join(tokens)
    return tokens
    

# Charger l'ensemble de données
#df = pd.read_csv('votre_fichier.csv')



In [24]:
# Appliquer la fonction Cleanup à tous les textes dans la colonne "text"
df2['text_process'] = df['text'].apply(text_processing)

In [25]:
df2.head()

,id,keyword,location,text,target,count,len_text,location_enc,text_process
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,1,69,0,deed reason earthquake may allah forgive u
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,1,38,0,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,1,133,0,resident asked 'shelter place notified officer...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,1,65,0,"13,000 people receive wildfire evacuation orde..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,1,88,0,got sent photo ruby alaska smoke wildfire pour...


In [26]:
df2.text_process[0]

'deed reason earthquake may allah forgive u'

In [27]:
X_train, X_test, y_train, y_test = train_test_split(df2['text_process'], df2['target'], test_size=0.2, random_state=42)


In [28]:
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [29]:
X_train_vec

<6090x17514 sparse matrix of type '<class 'numpy.int64'>'
	with 65731 stored elements in Compressed Sparse Row format>